In [ ]:
import pandas as pd
import re
from pathlib import Path

pd.options.display.max_colwidth = 600  # Set Full Display

# use r -- to avoid unicode error !
file_path=Path(r'C:\Users\LEON\austrump\澳盛销售交流群_2.xlsx')

In [ ]:
# file_path=Path(r'C:\Users\LEON\austrump\澳盛销售交流群_2.xlsx')

In [ ]:
# file_path=Path(r'C:\Users\LEON\austrump\诺兰地产房源群Norland Property.xlsx')

# Import Initial Data & Select Target Columns by Names:

In [ ]:

df = pd.read_excel(file_path, parse_dates=True)
df.dropna(inplace=True)
df_origin = df[['时间','联系人','微信号','消息']]
print(df_origin.info())

# Select 新格式开始执行 时间 之后的 Data:

In [ ]:
df_origin2=df_origin.loc[df_origin["时间"]>='2022-02-21 16:28:52']
print(df_origin2.info())
df_origin2.head(2)

# Filter Out Irrelevant Information:

In [ ]:
df_origin3=df_origin2[df_origin2['消息'].str.contains('房源描述')] 
print(df_origin3.info())

### Check & Drop Duplication:

In [ ]:
check=df_origin3["消息"].duplicated().sum()  # .item() convert numpy int to int
print(f"{check} 个 Duplicates") 

In [ ]:
df_origin4=df_origin3.drop_duplicates(subset="消息",keep='first')
print(df_origin4.info())

In [ ]:
df_origin4.head(5)

# Split I： 拆开 -- 然后查看  房源描述 的 Duplicates:

In [ ]:
df1= df_origin4['消息'].str.split('【房源描述】',expand=True)
df1.columns=["a","b"] # Create new name

print(df1.info())


### Check Null:

In [ ]:
df1[df1["b"].isnull()]


### b 列 中没有的值 copy a列的: 特别注意！

In [ ]:
df1.loc[df1["b"].isnull(), "b"]=df1["a"]
print(df1.info())


# Concat: pd.concat([df1, df2], axis=1): df_core3

In [ ]:
df_core=pd.concat([df_origin4,df1],axis=1)

print(df_core.info())
df_core.sample(3)

### Check Null:

In [ ]:
df_core[df_core["b"].isnull()]

PS: 【房源描述  --  所以没有被分割出来！！！！！！

In [ ]:

df_core2 = df_core.drop(columns=["消息"])
df_core2.columns=['时间','联系人','微信号','目标','房源描述']  #改名

print(df_core2.info())
df_core2.sample(2)



# 查看 房源描述 的 Duplicates: df_core3

In [ ]:
check2=df_core2["房源描述"].duplicated().sum()  # .item() convert numpy int to int
print(f"{check2} 个 Duplicates") 

In [ ]:
df_core3=df_core2.drop_duplicates(subset="房源描述")
print(df_core3.info())

# Split II： 把目标 弄出来

PS：无法 Extract 因为没有规律！！

Step 1: 先 Split 一部分 Extract 地址：

In [ ]:
df_core4= df_core3['目标'].str.split('【项目地址】',expand=True).rename(columns={0:"drop",1:"目标1",2:"目标2"}).drop(columns=["drop"])

print(df_core4.info())
df_core4.head()


# Split II：Check Missing Value Part I -- notNull: df_core4 目标1 --copy to 目标2

In [ ]:
df_core4[df_core4["目标2"].notnull()]

PS：index 34824 和 34976 目标位置不对，那我们把所有的值 copy 到 目标2

In [ ]:
df_core4.loc[df_core4["目标2"].isnull(),"目标2"]=df_core4["目标1"]
print(df_core4.info())
display(df_core4[df_core4["目标2"].notnull()].sample(3))

# Split II：Check missing value : df_core4


非常注意: 5个MISSSING （153 减去 148） !! 但无法在 df_core4内部 copy，需要对比 df_core3，留在 concat之后

In [ ]:
df_core4[df_core4["目标1"].isnull()]

Find missed value in original df:

PS: 非常注意 iloc or loc --- select index

PS: 而用 .index() -- row number

In [ ]:
df_core2.loc[[34836,34909,34910,34913,35550]]

Compare:

In [ ]:
df_core2.loc[[35738,34217]]

# Concat II: pd.concat([df1, df2], axis=1): df_core5

In [ ]:
df_core5=pd.concat([df_core3,df_core4],axis=1)

print(df_core5.info())


继续 Check Missing Value:

In [ ]:
df_core5[df_core5["目标2"].isnull()]

# Concat II: Missing Value Part II -- 目标 -- copy to 目标2, drop duplicates -- df_core6

In [ ]:
df_core5[df_core5["目标2"].notnull()].head(1)

In [ ]:
df_core5.loc[df_core5["目标2"].isnull(),"目标2"]=df_core5["目标"]
print(df_core5.info())

### 查看 目标2 的 Duplicates:

In [ ]:
check3=df_core5["目标2"].duplicated().sum()  # .item() convert numpy int to int
print(f"{check3} 个 Duplicates") 

In [ ]:
df_core6=df_core5.drop_duplicates(subset="目标2")
print(df_core6.info())

# Split III： 处理 目标2 再 分解： df_core7

In [ ]:
df_core7= df_core6['目标2'].str.split('【\S+】',expand=True).rename(columns={0:"地址",1:"信息1",2:"信息2",3:"信息3",4:"信息4",5:"信息5",6:"信息6"})

print(df_core7.info())
df_core7.head()

In [ ]:
# df_core7.loc[df_core7["信息6"].notnull()]

# Concat III: pd.concat([df1, df2], axis=1):

In [ ]:
df_final=pd.concat([df_core6,df_core7],axis=1)
print(df_final.info())


PS: 再 drop 目标，目标1，目标2，改变房源描述的位置：

PS: Summary:

1. 房源描述缺少一个，是因为有一个 【] 所以无法 split

2. 信息1 -- 信息6， 因为不规律， 无法 extract

# Clean Up: drop "目标"（已处理），改变column顺序：地址，房源描述：df_final3

In [ ]:
df_final2= df_final.drop(columns=["目标","目标1","目标2"])
print(df_final2.info())

In [ ]:
df_final3 = df_final2[["地址","时间","联系人","微信号","信息1","信息2","信息3","信息4","信息5","信息6","房源描述"]]
df_final3.sample(3)

# Check 残留 -- 信息1：中还有地址

In [ ]:
df_final3[df_final3["信息1"].str.contains("项目地址")]

In [ ]:
df_final3.loc[df_final3["信息1"].str.contains("项目地址"),"地址"]=df_final3["信息1"]

In [ ]:
df_final3[df_final3["信息1"].str.contains("项目地址")]

# Clean index column:

In [ ]:
df_final4 = df_final3.reset_index(drop=True)
df_final4.sample(1)

# 测试学习：

remove:
1. " * " 号   " 。"号  " | " 号 -- OK

2. 中文

3. " ( xxx ) " 包含  -- PS： 两种，英文括号，中文括号  -- OK

4. " * xxx * "  包含  -- OK 

drop: 地址中包含：
1. 空白 -- 没有英文的  -- 虽然空白被认为不是 null, 但不含英文肯定不是 -- OK

2. <> -- OK 

PS：测试 = 正确

In [ ]:
df_final4["地址"].str.contains("\*+").sum()

In [ ]:
df_final4[df_final4["地址"].str.contains("\*+")]

PS：测试 = 正确

In [ ]:
df_final4["地址"].str.contains("\。").sum()

In [ ]:
df_final4[df_final4["地址"].str.contains("\。+")]

PS：测试 = 正确

In [ ]:
df_final4["地址"].str.contains("\|+").sum()

In [ ]:
df_final4["地址"].str.contains(r"\(\D+\)").sum()

In [ ]:
df_final4[df_final4["地址"].str.contains("【")]

In [ ]:
df_final4[df_final4["地址"].str.contains("]")]

PS: <...> 表情乱码，测试 = 正确

In [ ]:
df_final4["地址"].str.contains(r"\<msg>").sum()

OR:

In [ ]:
df_final4["地址"].str.contains(r"\<title>").sum()

OR:

In [ ]:
df_final4[df_final4["地址"].str.contains(r"\<\S+>")]

PS: 中文 、 英文括号 = 5个 = 正确

In [ ]:
df_final4[df_final4["地址"].str.contains(r"\（\D+\）")]

In [ ]:
df_final4[df_final4["地址"].str.contains(r"\(\D+\)")]

PS: " * xxx  * "= 5个 = 正确

In [ ]:
df_final4[df_final4["地址"].str.contains(r"\*\D+\*")]

PS: 找到 空白 消息 = 正确！

非常注意：Map 会过滤掉，所以不要紧！！

非常注意：不包含 ！！！ 用 ~ ！！！ 

In [ ]:
df_final4[~df_final4["地址"].str.contains("[a-zA-Z0-9]")]

PS: 测试 是否包含中文：

In [ ]:
df_final4[df_final4["地址"].str.contains(u"[\u4e00-\u9fff]+")]

# Clean 地址中 怪异符号，Drop 错误信息：

In [ ]:
replace_char = r'\(\D+\）|\（\D+\)|\(\D+\)|\（\D+\）|\*|\。|\||\【|\】|\[|\]|\<\S+>|\*\D+\*|[\u4e00-\u9fff]'
replace_char

In [ ]:
df_final4["地址"]=df_final4["地址"].str.replace(replace_char,"")

# Check Again:

In [ ]:
df_final4["地址"].str.contains("\。+").sum()


In [ ]:
df_final4[df_final4["地址"].str.contains("\。+")]

In [ ]:
df_final4["地址"].str.contains("\|+").sum()

In [ ]:
df_final4["地址"].str.contains("\*+").sum()

In [ ]:
df_final4["地址"].str.contains("\【+").sum()

In [ ]:
df_final4["地址"].str.contains("\]+").sum()

In [ ]:
df_final4["地址"].str.contains(r"\<\S+>").sum()

In [ ]:
df_final4["地址"].str.contains(r"\（\D+\）").sum()

In [ ]:
df_final4["地址"].str.contains(r"\(\D+\)").sum()

In [ ]:
df_final4[df_final4["地址"].str.contains(r"\(\D+\)")]

In [ ]:
df_final4["地址"].str.contains(r"\*\D+\*").sum()

In [ ]:
df_final4["地址"].str.contains(u"[\u4e00-\u9fff]+").sum()

In [ ]:
df_final4[df_final4["地址"].str.contains(u"[\u4e00-\u9fff]+")]

# 根据 Map 要求， unit1/2xxx 需要空格 unit1/ 2xxxx

总共：

In [ ]:

df_final4["地址"].str.contains("/").sum()

有空格的：

In [ ]:
df_final4["地址"].str.contains("/ ").sum()

In [ ]:
df_final4["地址"].str.contains(" /").sum()

其中这个被算了 2次：

In [ ]:
df_final4["地址"].str.contains(" / ").sum()

完全没有空格的： 没有空格

In [ ]:
df_final4["地址"].str.contains(r"\S+/\S+").sum()

Replace:

In [ ]:
df_final4["地址"]=df_final4["地址"].str.replace("/","  /  ")

In [ ]:
df_final4["地址"].str.contains(r"\S+/\S+").sum()

# Inspection:

In [ ]:
# df_final4.sample(10)
df_final4.info()

# To Excel:

In [ ]:
# create excel writer object
writer=pd.ExcelWriter('Vicent_VerA_Output.xlsx')
# write dataframe to excel
df_final4.to_excel(writer, index=False)
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

# Summary:

注意：

1. 地址挤在一个 cell 肯定不行

2. 庄园，商铺，如果格式不对，也不行

3. 如果大型项目可以被 google 出来，可以不需要输入地址,比如 Deanside Attwell


1. Map 需要打开，然后人工点击每个 cell，就能读出地址

问题：
1. Map -- 多个点 在一个位置 

解决：

1. java 编辑 cluster

2. 直接查看边栏，(相似的地址已经集中) -- 但是！如果分享给别人看，他们看不到 LIST BAR 边栏！！！！